In [27]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Activation, Reshape
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, Adam, Adadelta
import numpy as np
import pandas as pd
from keras.utils import np_utils
import time
from keras.models import Sequential
import tempfile
from keras.models import save_model, load_model

In [28]:
train_data=pd.read_csv('train.csv')

In [29]:
train_data.head()

,label,feature
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [30]:
train_data.shape

(28709, 2)

In [31]:
pixel=train_data['feature'].values
label=train_data['label'].values

In [32]:
pixel.shape

(28709,)

In [33]:
label.shape

(28709,)

In [34]:
pixel[0]

'70 80 82 72 58 58 60 63 54 58 60 48 89 115 121 119 115 110 98 91 84 84 90 99 110 126 143 153 158 171 169 172 169 165 129 110 113 107 95 79 66 62 56 57 61 52 43 41 65 61 58 57 56 69 75 70 65 56 54 105 146 154 151 151 155 155 150 147 147 148 152 158 164 172 177 182 186 189 188 190 188 180 167 116 95 103 97 77 72 62 55 58 54 56 52 44 50 43 54 64 63 71 68 64 52 66 119 156 161 164 163 164 167 168 170 174 175 176 178 179 183 187 190 195 197 198 197 198 195 191 190 145 86 100 90 65 57 60 54 51 41 49 56 47 38 44 63 55 46 52 54 55 83 138 157 158 165 168 172 171 173 176 179 179 180 182 185 187 189 189 192 197 200 199 196 198 200 198 197 177 91 87 96 58 58 59 51 42 37 41 47 45 37 35 36 30 41 47 59 94 141 159 161 161 164 170 171 172 176 178 179 182 183 183 187 189 192 192 194 195 200 200 199 199 200 201 197 193 111 71 108 69 55 61 51 42 43 56 54 44 24 29 31 45 61 72 100 136 150 159 163 162 163 170 172 171 174 177 177 180 187 186 187 189 192 192 194 195 196 197 199 200 201 200 197 201 137 58 98 92

In [35]:
type(pixel)

numpy.ndarray

In [36]:
pixels=[ np.fromstring(x, dtype=float, sep=' ') for x in pixel ]

In [37]:
pixels[0]

array([  70.,   80.,   82., ...,  106.,  109.,   82.])

In [38]:
pixels=np.array(pixels)
pixels=pixels.reshape(pixels.shape[0],48*48)
pixels=pixels.astype('float32')
pixels=pixels/255

In [39]:
pixels.shape

(28709, 2304)

In [40]:
# 将类向量转换为二进制类矩阵
label = np_utils.to_categorical(label, 7)
label.shape

(28709, 7)

In [41]:
#分成训练集和测试集
id=int(pixels.shape[0]*0.8)
X_train=pixels[:id]
X_test=pixels[id:]
y_train=label[:id]
y_test=label[id:]

In [42]:
X_train.shape

(22967, 2304)

In [43]:
X_test.shape

(5742, 2304)

In [44]:
y_train.shape

(22967, 7)

In [51]:
# 定义超参数
nb_epoch=120
batch_size = 100
units=1024
nb_classes=7


model=Sequential()
model.add(Dense(input_dim=48*48,units=units,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=units,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=units,activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# 模型编译
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 526)               1212430   
_________________________________________________________________
dropout_19 (Dropout)         (None, 526)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 526)               277202    
_________________________________________________________________
dropout_20 (Dropout)         (None, 526)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 526)               277202    
_________________________________________________________________
dropout_21 (Dropout)         (None, 526)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 526)               277202    
__________

In [52]:
# 模型训练
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,validation_data=(X_test, y_test))

# 模型评估或模型预测
score = model.evaluate(X_train, y_train)
print('\nTrain score:', score[0])  # 损失值  Test score: 
print('Train accuracy:', score[1])  # 准确率  Test accuracy: 
score = model.evaluate(X_test, y_test,)
print('\nTest score:', score[0])  # 损失值  Test score: 
print('Test accuracy:', score[1])  # 准确率  Test accuracy: 


Train on 22967 samples, validate on 5742 samples
Epoch 1/120
22967/22967 [==============================] - 3s - loss: 1.8671 - acc: 0.2244 - val_loss: 1.8556 - val_acc: 0.2517
Epoch 2/120
22967/22967 [==============================] - 2s - loss: 1.8267 - acc: 0.2460 - val_loss: 1.8114 - val_acc: 0.2517
Epoch 3/120
22967/22967 [==============================] - 2s - loss: 1.8154 - acc: 0.2501 - val_loss: 1.8115 - val_acc: 0.2517
Epoch 4/120
22967/22967 [==============================] - 2s - loss: 1.8046 - acc: 0.2509 - val_loss: 1.8092 - val_acc: 0.2517
Epoch 5/120
22967/22967 [==============================] - ETA: 0s - loss: 1.7979 - acc: 0.250 - 2s - loss: 1.7980 - acc: 0.2509 - val_loss: 1.7750 - val_acc: 0.2665
Epoch 6/120
22967/22967 [==============================] - 2s - loss: 1.7870 - acc: 0.2571 - val_loss: 1.7719 - val_acc: 0.2762
Epoch 7/120
22967/22967 [==============================] - 2s - loss: 1.7804 - acc: 0.2626 - val_loss: 1.7692 - val_acc: 0.2698
Epoch 8/120
22967

22967/22967 [==============================] - 2s - loss: 1.6669 - acc: 0.3438 - val_loss: 1.7102 - val_acc: 0.3206
Epoch 64/120
22967/22967 [==============================] - 2s - loss: 1.6674 - acc: 0.3448 - val_loss: 1.6967 - val_acc: 0.3370
Epoch 65/120
22967/22967 [==============================] - 2s - loss: 1.6681 - acc: 0.3418 - val_loss: 1.6757 - val_acc: 0.3638
Epoch 66/120
22967/22967 [==============================] - 2s - loss: 1.6704 - acc: 0.3391 - val_loss: 1.6939 - val_acc: 0.3488
Epoch 67/120
22967/22967 [==============================] - 2s - loss: 1.6690 - acc: 0.3450 - val_loss: 1.7014 - val_acc: 0.3365
Epoch 68/120
22967/22967 [==============================] - 2s - loss: 1.6594 - acc: 0.3485 - val_loss: 1.6637 - val_acc: 0.3727
Epoch 69/120
22967/22967 [==============================] - 2s - loss: 1.6605 - acc: 0.3469 - val_loss: 1.6817 - val_acc: 0.3553
Epoch 70/120
22967/22967 [==============================] - 2s - loss: 1.6589 - acc: 0.3481 - val_loss: 1.7156

In [47]:
# 保存模型
_, fname = tempfile.mkstemp('222.h5', dir='.')
save_model(model, fname)
new_model = load_model(fname)

In [48]:
fname='222.h5'
new_model = load_model(fname)
# 画模型图
from keras.utils.vis_utils import plot_model
plot_model(new_model,show_shapes=True,to_file='model.png')

OSError: Unable to open file (Unable to open file: name = '222.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
# 用训练好的模型预测未知图片的情感
test_data=pd.read_csv('test.csv')

In [ ]:
test_data.head()

In [ ]:
test_data.shape

In [ ]:
test_id=test_data['id'].values
test_pixel=test_data['feature'].values

In [ ]:
test_pixels=[ np.fromstring(x, dtype=float, sep=' ') for x in test_pixel ]
test_pixels=np.array(test_pixels)
test_pixels=test_pixels.reshape(test_pixels.shape[0],48,48,1)
test_pixels=test_pixels.astype('float32')
test_pixels=test_pixels/255

In [ ]:
test_pixels.shape

In [ ]:
y_pred=new_model.predict(test_pixels)

In [ ]:
y_pred[2][:]

In [ ]:
def find(x):
    count=0
    maxnum=0
    maxcount=0
    for i in x:
        if maxnum<i:
            maxcount=count
            maxnum=i
        count=count+1
    return maxcount
y_label=[find(x) for x in y_pred]

In [ ]:
y_label=np.array(y_label)

In [ ]:
y_label.shape

In [ ]:
y_label[46]

In [ ]:
pd.DataFrame({"id": test_id, "label": y_label}).to_csv('submission.csv',index=False)